<a href="https://colab.research.google.com/github/AbdullahNasser98/Tahaluf-project/blob/master/Task-2%20(Cloth%20classification)/rf_calculation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from tensorflow import keras

In [1]:
## Clone repo to our work space
! git clone https://github.com/google-research/receptive_field.git

Cloning into 'receptive_field'...
remote: Enumerating objects: 79, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 79 (delta 4), reused 4 (delta 4), pack-reused 72
Unpacking objects: 100% (79/79), done.


In [16]:
# Mound drive to read the model
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [20]:
cnn_model = keras.models.load_model('/content/gdrive/MyDrive/yolox_weights/clothing_CNN_model.hdf5')

Key is the name of the layer and the value is the array consisting of kernel size, stride and padding respectively. This means that layer conv2 has 5×5 kernel, 1×1 striding and 2×2 padding.
`'conv2': [5, 1, 2]`

In [34]:
# define conv and dense layers parameters
#kernel, stride, padding
cnn = {
    'conv1': [2, 1, 0],
    'pool1': [2, 1, 0],
    'conv2': [2, 1, 0],
    'pool2': [2, 1, 0],
    'conv3': [2, 1, 0],
    'pool3': [2, 1, 0],
    'conv4': [2, 1, 0],
    'pool4': [2, 1, 0],
    'fc6-conv': [128, 1, 0],
    'fc7-conv': [10, 1, 0]
}

In [35]:
# Function to calculate RF
import math;

class ReceptiveFieldCalculator():
    def calculate(self, architecture, input_image_size):
        input_layer = ('input_layer', input_image_size, 1, 1, 0.5)
        self._print_layer_info(input_layer)
        
        for key in architecture:
            current_layer = self._calculate_layer_info(architecture[key], input_layer, key)
            self._print_layer_info(current_layer)
            input_layer = current_layer
            
    def _print_layer_info(self, layer):
        print(f'------')
        print(f'{layer[0]}: n = {layer[1]}; r = {layer[2]}; j = {layer[3]}; start = {layer[4]}')     
        print(f'------')
            
    def _calculate_layer_info(self, current_layer, input_layer, layer_name):
        n_in = input_layer[1]
        j_in = input_layer[2]
        r_in = input_layer[3]
        start_in = input_layer[4]
        
        k = current_layer[0]
        s = current_layer[1]
        p = current_layer[2]

        n_out = math.floor((n_in - k + 2*p)/s) + 1
        padding = (n_out-1)*s - n_in + k 
        p_right = math.ceil(padding/2)
        p_left = math.floor(padding/2)

        j_out = j_in * s
        r_out = r_in + (k - 1)*j_in
        start_out = start_in + ((k-1)/2 - p_left)*j_in
        return layer_name, n_out, j_out, r_out, start_out

In [36]:
calculator = ReceptiveFieldCalculator()
calculator.calculate(cnn, 256)

------
input_layer: n = 256; r = 1; j = 1; start = 0.5
------
------
conv1: n = 255; r = 1; j = 2; start = 1.0
------
------
pool1: n = 254; r = 1; j = 3; start = 1.5
------
------
conv2: n = 253; r = 1; j = 4; start = 2.0
------
------
pool2: n = 252; r = 1; j = 5; start = 2.5
------
------
conv3: n = 251; r = 1; j = 6; start = 3.0
------
------
pool3: n = 250; r = 1; j = 7; start = 3.5
------
------
conv4: n = 249; r = 1; j = 8; start = 4.0
------
------
pool4: n = 248; r = 1; j = 9; start = 4.5
------
------
fc6-conv: n = 121; r = 1; j = 136; start = 68.0
------
------
fc7-conv: n = 112; r = 1; j = 145; start = 72.5
------


This model's rf is 1 because we have the same stride=1, padding='valid', and kernel=2, constant across all our layers